In [1]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt         
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint
%matplotlib inline
from tensorflow.keras import models, layers, regularizers, Input
plt.rcParams["axes.grid"] = False
import keras

In [2]:
class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

In [3]:
def model_plots(history,name):
    plt.style.use('ggplot')
    
    fig1, ax1 = plt.subplots(figsize=(6,6))
    metrics = ['acc','val_acc']
    for metric in metrics:
        ax1 = plt.plot(history.history[metric], label=metric)
    ax1 = plt.title(f'{name}-Accuracy')

    ax1 = plt.legend()
    ax1 = plt.tight_layout()
    ax1 = plt.xlabel('Epochs')
    ax1 = plt.ylabel('Accuracy')
    fig1.savefig(f'Images/metrics/{name}-accuracy.jpg',bbox_inches='tight', dpi=150)
    
    fig2, ax2 = plt.subplots(figsize=(6,6))
    metrics = ['loss','val_loss']
    for metric in metrics:
        ax2 = plt.plot(history.history[metric], label=metric)
    ax2 = plt.title(f'{name}-Loss')

    ax2 = plt.legend()
    ax2 = plt.tight_layout()
    ax2 = plt.xlabel('Epochs')
    ax2 = plt.ylabel('Loss')
    fig2.savefig(f'Images/metrics/{name}-loss.jpg',bbox_inches='tight', dpi=150)

In [4]:
def load_data(dims):
    """
        Load the data:
            - 14,034 images to train the network.
            - 3,000 images to evaluate how accurately the network learned to classify images.
    """
    
    datasets = ['../../../../archive/seg_train/seg_train', '../../../../archive/seg_test/seg_test']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, dims) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [5]:
(train_images_resized_75, train_labels), (test_images_resized_75, test_labels) = load_data((75,75))
(train_images_resized_150, train_labels), (test_images_resized_150, test_labels) = load_data((150,150))

  3%|▎         | 57/2191 [00:00<00:03, 566.10it/s]

Loading ../../../../archive/seg_train/seg_train


 17%|█▋        | 75/437 [00:00<00:00, 744.35it/s]

Loading ../../../../archive/seg_test/seg_test


  8%|▊         | 168/2191 [00:00<00:01, 1664.24it/s]

Loading ../../../../archive/seg_train/seg_train


 68%|██████▊   | 296/437 [00:00<00:00, 1377.48it/s]

Loading ../../../../archive/seg_test/seg_test


100%|██████████| 501/501 [00:00<00:00, 1606.69it/s]


In [6]:
train_images_resized_scaled_75 =train_images_resized_75.reshape((-1,75,75,3)).astype('float32') / 255.0
test_images_resized_scaled_75 = test_images_resized_75.reshape((-1,75,75,3)).astype('float32') / 255.0

train_images_resized_scaled_150 = train_images_resized_150.reshape((-1,150,150,3)).astype('float32') / 255.0
test_images_resized_scaled_150 = test_images_resized_150.reshape((-1,150,150,3)).astype('float32') / 255.0

In [7]:
models_path = '../../../../Models'

def load_models(path):
    '''Creates a tuple of filename and model
    Takes folder path containing models'''
    mods = []
    #get each saved model and append it to list mods
    for file in tqdm(os.listdir(os.path.join(path))):
        
        #model path
        model_path = os.path.join(os.path.join(path), file)

        mods.append((file,keras.models.load_model(model_path)))
    return mods


In [8]:
mods = load_models(models_path)

100%|██████████| 15/15 [00:08<00:00,  1.80it/s]


In [9]:
# mod0_hist = mods[0][1].evaluate(train_images_resized_scaled,train_labels,return_dict=True)

In [10]:
# test_clustered = np.load('../../../../Datasets/clusted_images_test.npz')['args']
# train_clustered = np.load('../../../../Datasets/clusted_images_train.npz')['args']
# test_clustered_labels = np.load('../../../../Datasets/test_labels.npz')['args']
# train_clustered_labels = np.load('../../../../Datasets/train_labels.npz')['args']

In [20]:
mods[0][1].summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 73, 73, 512)       14336     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 512)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 24, 24, 512)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 22, 22, 156)       719004    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 20, 20, 156)       219180    
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 20, 156)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 18, 18, 128)      

In [22]:
def model_scorer(models,train_eval_imgs_75,train_eval_imgs_150,train_eval_labels,test_eval_imgs_75,test_eval_imgs_150,test_eval_labels):
    '''Takes a list of loaded models and evaluates them on the evaluation images and labels passed
    returns list of tuples containing the model name and a dict of loss and accuracy for each model, can pass different sized images'''
    model_scores = []
    
    for model_tup in models:
        print(model_tup[0])
        model_dict = {}
        try:
            
            try:
                model_dict['train'][model_tup[0].astype(str)] = model_tup[1].evaluate(train_eval_imgs_75,train_eval_labels,return_dict=True)
                model_dict['test'][model_tup[0].astype(str)] = model_tup[1].evaluate(test_eval_imgs_75,test_eval_labels,return_dict=True)
            
                model_scores.append(models_dict)
#             model_scores.append((model_tup[0],model_tup[1].evaluate(eval_imgs_75,eval_labels,return_dict=True)))

            except:
        
                model_dict['train'][model_tup[0].astype(str)] = model_tup[1].evaluate(train_eval_imgs_150,train_eval_labels,return_dict=True)
                model_dict['test'][model_tup[0].astype(str)] = model_tup[1].evaluate(test_eval_imgs_150,test_eval_labels,return_dict=True)
            
                model_scores.append(models_dict)
        
        except:
            
            continue  

#             model_scores.append((model_tup[0],model_tup[1].evaluate(eval_imgs_150,eval_labels,return_dict=True)))
        
    return model_scores

In [23]:
scores = model_scorer(mods,
                      train_images_resized_scaled_75,train_images_resized_scaled_150,train_labels,
                      test_images_resized_scaled_75,test_images_resized_scaled_150,test_labels)

addedL1reg-withmaxpool_model.h5
439/439 [==============================] - 115s 262ms/step - loss: 9.5569 - acc: 0.2012
baseline_dropout_model.h5
439/439 [==============================] - 36s 81ms/step - loss: 320.2752 - acc: 0.3349
baseline_model.h5
439/439 [==============================] - 51s 115ms/step - loss: 0.2421 - acc: 0.9273
L2-reg_2nd-dropout_model.h5
439/439 [==============================] - 26s 59ms/step - loss: 0.5860 - acc: 0.7963
L2-reg_model.h5
439/439 [==============================] - 26s 59ms/step - loss: 0.5044 - acc: 0.8378
loaded_model_test_model.h5
439/439 [==============================] - 18s 40ms/step - loss: 0.7391 - acc: 0.7257
more_epochs_for_lime_model.h5
439/439 [==============================] - 26s 59ms/step - loss: 0.4317 - acc: 0.1618
more_reg_model.h5
439/439 [==============================] - 26s 59ms/step - loss: 0.6685 - acc: 0.1341
resized_dropout_model.h5
439/439 [==============================] - 10s 22ms/step - loss: 0.3592 - acc: 0.8773
r

In [24]:
scores

[]

In [ ]:
import lime

In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries

In [ ]:
import random

In [ ]:
def image_expl(model,image_array,name,image_label):
    '''
    Takes a model, single image array, name string of the model, and the true label of the image.
    Uses LIME to get an image explainer and plots the image and explaination.
    '''
    
    fig, ax = plt.subplots()
    #instantiate explainer
    explainer = lime_image.LimeImageExplainer(random_state=1)
    #explain an image caste to double bc updated keras version, pass predictor 
    explanation = explainer.explain_instance(image_array.astype('double'), model.predict)
    #show the image
    ax = plt.imshow(image_array)
    #get the mask for the prediction
    temp, mask = explanation.get_image_and_mask(model.predict(image_array.reshape((1,75,75,3))).argmax(axis=1)[0], positive_only=True, hide_rest=False)
    #show image and mask together
    ax = plt.imshow(mark_boundaries(temp, mask))
    
    ax = plt.title(f'{image_label}-{name}')
    


    fig.savefig(f'Images/explainer/{name}-expl.jpg',bbox_inches='tight', dpi=150)

In [ ]:
image_expl(model,train_images_resized_scaled[1500],'train-1500',train_labels[1500])

In [ ]:
def image_preds(model,image_array,name,image_label):
    '''
    Takes a model, single image array, name string of the model, and the true label of the image.
    Plots the class predictions for the image.
    '''
    plt.style.use('fivethirtyeight')

    fig, ax = plt.subplots(figsize=(6,6))
    ax = plt.plot(range(0,6),model.predict(image_array.reshape((1,75,75,3)))[0])
    ax = plt.xticks(ticks=[0,1,2,3,4,5],labels=class_names)
    ax = plt.tight_layout()
    ax = plt.xlabel('Class Name')
    ax = plt.ylabel('Confidence')
    ax = plt.title(f'{image_label}-{name}')
    fig.savefig(f'Images/preds/{name}-preds.jpg',bbox_inches='tight', dpi=150)

In [ ]:
image_preds(model,train_images_resized_scaled[1500],'train-1500',train_labels[1500])

In [ ]:
# rand_img = np.random.randint(0,14034,10)
# for num in rand_img:
#     label = train_labels[num]
#     image_expl(model,train_images_resized_scaled[num],f'train-{num}-expl',label)
#     image_preds(model,train_images_resized_scaled[num],f'train-{num}-pred',label)